In [8]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

### LDA를 사용한다는 아이디어 자체가 등급별로 보고서 글의 특징이 있을 것이라 생각
#### -> 각각의 보고서를 쪼개야함 

In [4]:
# 모든 문서의 내용을 한 문장으로 합치고 관리하기 위해 E, S, G 리스트를 만듬
E_list = []
S_list = []
G_list = []

# E, S, G 등급별로 보고서 내용을 분류해야 하기 때문에 1~9등급을 상징하도록 9개의 리스트를 넣음 
# E 리스트의 index=0 의 리스트는 E등급이 1등급을 받은 보고서의 E관련 보고서 내용이 들어감 
for i in range(9):
    E_list.append([])
    S_list.append([])
    G_list.append([])

In [6]:
report_name_list= [] # 나중에 이부분은 전처리 코드가 완료되면 따로 할당 
pre_dict = {} # 위와 동일 
    
for i in report_name_list:
    e_label = pre_dict[i][3]['label'][0]
    s_label = pre_dict[i][3]['label'][1]
    g_label = pre_dict[i][3]['label'][2]
    t_label = pre_dict[i][3]['label'][3]
    
    E[e_label-1] += pre_dict[i][0]['E']
    E[s_label-1] += pre_dict[i][1]['S']
    E[g_label-1] += pre_dict[i][2]['G']

명사를 추출하기 위해서 konlpy를 활용해서 불필요한 조사, 공통적으로 반복되는 명사, 기호들을 모아서 불용어(stopwords) 리스트를 만들어서 제거하도록 하였음 맨 아래에 불용어 리스트를 정의해 두었음 }

In [7]:
from konlpy.tag import Twitter
def tokenize(doc) : 
    post_tagger = Twitter( )
    noun_list = post_tagger.nouns(doc)
    final_noun_list = [''.join(t) for t in noun_list if t not in stop_keyword]
    return final_noun_list

In [ ]:
final_E_list = []
final_S_list = []
final_G_list = []

for i in range(9):
    final_E_list.insert(i,tokenize(E_list[i]))
    final_S_list.insert(i,tokenize(S_list[i]))
    final_G_list.insert(i,tokenize(G_list[i]))

In [ ]:
train_total_tfid_vect=[]
total_lda_result=[]
for i in range(len(final_E_list)):
    # Tfidfvectorizer 변환
    tfidf_vect=TfidfVectorizer(ngram_range=(1,2),max_df=300)
    train_total_tfid_vect.insert(len(train_total_tfid_vect),tfidf_vect)
    tfidf_vect=tfidf_vect.fit_transform(train_total[i])
    
    # lda 모델에 입력 
    lda_train = LatentDirichletAllocation(n_components = 1, random_state = 0)
    lda_train.fit(tfidf_vect)
    total_lda_result.insert(len(total_lda_result),lda_train)
    
print(len(train_total_tfid_vect))
print(len(total_lda_result))  

lda 모델링의 결과물인 키워드를 리스트에 저장하고 print해주는 함수 

In [9]:
def display_and_save_topics(model, feature_names, no_top_words, lst) :
    for topic in model.components_ :
        
        topic_word_indexes = topic.argsort()[::-1]
        top_indexes = topic_word_indexes[:no_top_words]
        
        feature_concat = ' '.join([feature_names[i] for i in top_indexes])
        print(feature_concat)
        lst.append(feature_concat)

tfidf변환을 해주고 lda 모델링을 한 결과물을 total_str_list에 index는 앞과 같은 순서로 저장 및 출력을 해줌 

In [ ]:
E_grade_list=['1','2','3','4','5','6','7','8','9']

In [ ]:
total_feature_name=[]
total_str_list=[]

for i in range(len(E_grade_list)):
    lst=[]
    feature_name=train_total_tfid_vect[i].get_feature_names()
    print(f"----{E_grade_list[i]}----\n")
    display_and_save_topics(total_lda_result[i], feature_name, 10, lst)
    total_str_list.insert(len(total_str_list),lst[0])

## Test

테스트 세트의 보고서를 1개씩 불러들여 개별로 LDA 토픽 모델링을 수행한 후 주요 단어 30개를 추출함 

테스트 보고서로부터 추출한 주요단어 30개를 앞서 범죄유형별 문서집합 LDA 토픽 모델링을 통해 추출한 보고서별 단어그룹들과 차례로 결합하여 tfidfvectorizer를 이용해 벡터 스페이스 모델로 변환 

변환한 테스트 벡터 스페이스 데이터를 lda모델에 학습 키워드를 추출한 후 str화를 진행하고 str과 앞서 train_data로 만들어놓은 각 보고서별 토픽 str을 리스트에 저장 

리스트에 그럼 ['1','2', ... ,'9'] 으로 이루어져있으므로 코사인 유사도 검사 가능 따라서 코사인 유사도 검사를 진행한다. 

## 유틸함수

In [22]:
stop_keyword = """
아
휴
아이구
아이쿠
아이고
어
나
우리
저희
따라
의해
을
를
에
의
가
으로
로
에게
뿐이다
의거하여
근거하여
입각하여
기준으로
예하면
예를 들면
예를 들자면
저
소인
소생
저희
지말고
하지마
하지마라
다른
물론
또한
그리고
비길수 없다
해서는 안된다
뿐만 아니라
만이 아니다
만은 아니다
막론하고
관계없이
그치지 않다
그러나
그런데
하지만
든간에
논하지 않다
따지지 않다
설사
비록
더라도
아니면
만 못하다
하는 편이 낫다
불문하고
향하여
향해서
향하다
쪽으로
틈타
이용하여
타다
오르다
제외하고
이 외에
이 밖에
하여야
비로소
한다면 몰라도
외에도
이곳
여기
부터
기점으로
따라서
할 생각이다
하려고하다
이리하여
그리하여
그렇게 함으로써
하지만
일때
할때
앞에서
중에서
보는데서
으로써
로써
까지
해야한다
일것이다
반드시
할줄알다
할수있다
할수있어
임에 틀림없다
한다면
등
등등
제
겨우
단지
다만
할뿐
딩동
댕그
대해서
대하여
대하면
훨씬
얼마나
얼마만큼
얼마큼
남짓
여
얼마간
약간
다소
좀
조금
다수
몇
얼마
지만
하물며
또한
그러나
그렇지만
하지만
이외에도
대해 말하자면
뿐이다
다음에
반대로
반대로 말하자면
이와 반대로
바꾸어서 말하면
바꾸어서 한다면
만약
그렇지않으면
까악
툭
딱
삐걱거리다
보드득
비걱거리다
꽈당
응당
해야한다
에 가서
각
각각
여러분
각종
각자
제각기
하도록하다
와
과
그러므로
그래서
고로
한 까닭에
하기 때문에
거니와
이지만
대하여
관하여
관한
과연
실로
아니나다를가
생각한대로
진짜로
한적이있다
하곤하였다
하
하하
허허
아하
거바
와
오
왜
어째서
무엇때문에
어찌
하겠는가
무슨
어디
어느곳
더군다나
하물며
더욱이는
어느때
언제
야
이봐
어이
여보시오
흐흐
흥
휴
헉헉
헐떡헐떡
영차
여차
어기여차
끙끙
아야
앗
아야
콸콸
졸졸
좍좍
뚝뚝
주룩주룩
솨
우르르
그래도
또
그리고
바꾸어말하면
바꾸어말하자면
혹은
혹시
답다
및
그에 따르는
때가 되어
즉
지든지
설령
가령
하더라도
할지라도
일지라도
지든지
몇
거의
하마터면
인젠
이젠
된바에야
된이상
만큼	어찌됏든
그위에
게다가
점에서 보아
비추어 보아
고려하면
하게될것이다
일것이다
비교적
좀
보다더
비하면
시키다
하게하다
할만하다
의해서
연이서
이어서
잇따라
뒤따라
뒤이어
결국
의지하여
기대여
통하여
자마자
더욱더
불구하고
얼마든지
마음대로
주저하지 않고
곧
즉시
바로
당장
하자마자
밖에 안된다
하면된다
그래
그렇지
요컨대
다시 말하자면
바꿔 말하면
즉
구체적으로
말하자면
시작하여
시초에
이상
허
헉
허걱
바와같이
해도좋다
해도된다
게다가
더구나
하물며
와르르
팍
퍽
펄렁
동안
이래
하고있었다
이었다
에서
로부터
까지
예하면
했어요
해요
함께
같이
더불어
마저
마저도
양자
모두
습니다
가까스로
하려고하다
즈음하여
다른
다른 방면으로
해봐요
습니까
했어요
말할것도 없고
무릎쓰고
개의치않고
하는것만 못하다
하는것이 낫다
매
매번
들
모
어느것
어느
로써
갖고말하자면
어디
어느쪽
어느것
어느해
어느 년도
라 해도
언젠가
어떤것
어느것
저기
저쪽
저것
그때
그럼
그러면
요만한걸
그래
그때
저것만큼
그저
이르기까지
할 줄 안다
할 힘이 있다
너
너희
당신
어찌
설마
차라리
할지언정
할지라도
할망정
할지언정
구토하다
게우다
토하다
메쓰겁다
옆사람
퉤
쳇
의거하여
근거하여
의해
따라
힘입어
그
다음
버금
두번째로
기타
첫번째로
나머지는
그중에서
견지에서
형식으로 쓰여
입장에서
위해서
단지
의해되다
하도록시키다
뿐만아니라
반대로
전후
전자
앞의것
잠시
잠깐
하면서
그렇지만
다음에
그러한즉
그런즉
남들
아무거나
어찌하든지
같다
비슷하다
예컨대
이럴정도로
어떻게
만약
만일
위에서 서술한바와같이
인 듯하다
하지 않는다면
만약에
무엇
무슨
어느
어떤
아래윗
조차
한데
그럼에도 불구하고
여전히
심지어
까지도
조차도
하지 않도록
않기 위하여
때
시각
무렵
시간
동안
어때
어떠한
하여금
네
예
우선
누구
누가 알겠는가
아무도
줄은모른다
줄은 몰랏다
하는 김에
겸사겸사
하는바
그런 까닭에
한 이유는
그러니
그러니까
때문에
그
너희
그들
너희들
타인
것
것들
너
위하여
공동으로
동시에
하기 위하여
어찌하여
무엇때문에
붕붕
윙윙
나
우리
엉엉
휘익
윙윙
오호
아하
어쨋든
만 못하다	하기보다는
차라리
하는 편이 낫다
흐흐
놀라다
상대적으로 말하자면
마치
아니라면
쉿
그렇지 않으면
그렇지 않다면
안 그러면
아니었다면
하든지
아니면
이라면
좋아
알았어
하는것도
그만이다
어쩔수 없다
하나
일
일반적으로
일단
한켠으로는
오자마자
이렇게되면
이와같다면
전부
한마디
한항목
근거로
하기에
아울러
하지 않도록
않기 위해서
이르기까지
이 되다
로 인하여
까닭으로
이유만으로
이로 인하여
그래서
이 때문에
그러므로
그런 까닭에
알 수 있다
결론을 낼 수 있다
으로 인하여
있다
어떤것
관계가 있다
관련이 있다
연관되다
어떤것들
에 대해
이리하여
그리하여
여부
하기보다는
하느니
하면 할수록
운운
이러이러하다
하구나
하도다
다시말하면
다음으로
에 있다
에 달려 있다
우리
우리들
오히려
하기는한데
어떻게
어떻해
어찌됏어
어때
어째서
본대로
자
이
이쪽
여기
이것
이번
이렇게말하자면
이런
이러한
이와 같은
요만큼
요만한 것
얼마 안 되는 것
이만큼
이 정도의
이렇게 많은 것
이와 같다
이때
이렇구나
것과 같이
끼익
삐걱
따위
와 같은 사람들
부류의 사람들
왜냐하면
중의하나
오직
오로지
에 한하다
하기만 하면
도착하다
까지 미치다
도달하다
정도에 이르다
할 지경이다
결과에 이르다
관해서는
여러분
하고 있다
한 후
혼자
자기
자기집
자신
우에 종합한것과같이
총적으로 보면
총적으로 말하면
총적으로
대로 하다
으로서
참
그만이다
할 따름이다
쿵
탕탕
쾅쾅
둥둥
봐
봐라
아이야
아니
와아
응
아이
참나
년
월
일
령
영
일
이
삼
사
오
육
륙
칠
팔
구
이천육
이천칠
이천팔
이천구
하나
둘
셋
넷
다섯
여섯
일곱
여덟
아홉
령
영
노
통
부
제
조
항
등
누구
상태
법
규정
외
대한
뿐
그
호
검사
서울
북부
지법
선고
판결
원심
사건
지방
법원
합의
환송
규정
이상
상태
같은
피고인
이
사실
것
위
점
수
피해자
이유
범죄
공소
증거
진술
및
처벌
부분
위반
행위
죄
작
법률
경우
위법
정도
기재
형법
결과
판단
기록
의
유죄
형
상고
당시
후
수사
중
경
년
범행
주장
경찰
가중
원
법정
무죄
때
시간
바
만
불
사정
경찰관
절차
취지
징역
요지
다음
도
특정
법리
현장
항소
내용
자신
업무
감정
약
앞
조피
쪽
영향
사람
조의
현수
이정훈
나
수원역
적법
대법원
금고
영장
방법
심
기각
재판
심리
일
자백
치
파기
주문
회
고등법원
고소
증인
공판
로서
작성
함
변호사
대법관
논
의견
법관
신병
면
검찰
일치
관여
것이므
다시
산업
양형
변호인
변경
구금
동
모두
부당
동녀
법칙
사유
따라서
알
음
진술조서
전
소론
심의
차
시
세
채
경위
범인
과정
정당
여부
증언
채증
공소장
옷
기관
인정
소재
사법
시한
명백
증거능력
본형
참조
국가보안법
대한민국
의사
강
경합
청구인
징역형
조사
"""
stop_keyword = stop_keyword.replace('\n', ' ')
stop_keyword_lst = stop_keyword.split()
stop_keyword_lst

['아',
 '휴',
 '아이구',
 '아이쿠',
 '아이고',
 '어',
 '나',
 '우리',
 '저희',
 '따라',
 '의해',
 '을',
 '를',
 '에',
 '의',
 '가',
 '으로',
 '로',
 '에게',
 '뿐이다',
 '의거하여',
 '근거하여',
 '입각하여',
 '기준으로',
 '예하면',
 '예를',
 '들면',
 '예를',
 '들자면',
 '저',
 '소인',
 '소생',
 '저희',
 '지말고',
 '하지마',
 '하지마라',
 '다른',
 '물론',
 '또한',
 '그리고',
 '비길수',
 '없다',
 '해서는',
 '안된다',
 '뿐만',
 '아니라',
 '만이',
 '아니다',
 '만은',
 '아니다',
 '막론하고',
 '관계없이',
 '그치지',
 '않다',
 '그러나',
 '그런데',
 '하지만',
 '든간에',
 '논하지',
 '않다',
 '따지지',
 '않다',
 '설사',
 '비록',
 '더라도',
 '아니면',
 '만',
 '못하다',
 '하는',
 '편이',
 '낫다',
 '불문하고',
 '향하여',
 '향해서',
 '향하다',
 '쪽으로',
 '틈타',
 '이용하여',
 '타다',
 '오르다',
 '제외하고',
 '이',
 '외에',
 '이',
 '밖에',
 '하여야',
 '비로소',
 '한다면',
 '몰라도',
 '외에도',
 '이곳',
 '여기',
 '부터',
 '기점으로',
 '따라서',
 '할',
 '생각이다',
 '하려고하다',
 '이리하여',
 '그리하여',
 '그렇게',
 '함으로써',
 '하지만',
 '일때',
 '할때',
 '앞에서',
 '중에서',
 '보는데서',
 '으로써',
 '로써',
 '까지',
 '해야한다',
 '일것이다',
 '반드시',
 '할줄알다',
 '할수있다',
 '할수있어',
 '임에',
 '틀림없다',
 '한다면',
 '등',
 '등등',
 '제',
 '겨우',
 '단지',
 '다만',
 '할뿐',
 '딩동',
 '댕그',
 '대해서',
 '대하여',
 '대하면',
